In [6]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

file = glob.glob("train/train.tsv")
pd.read_csv("train/train.tsv", delimiter='\t')
data_list = []
data_list.append(pd.read_csv("train/train.tsv", delimiter='\t', index_col=0))
df = pd.concat(data_list)
#df.info()

In [2]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
df_train, df_val = train_test_split(df, test_size=0.2)
# print(type(df))
# print(type(df_train))
# print(type(df_val))

col = "Cover_Type"

train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids,
                  num_boost_round=1000,
                  callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)])


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001318 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2269
[LightGBM] [Info] Number of data points in the train set: 232404, number of used features: 53
[LightGBM] [Info] Start training from score 2.052968
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 0.35067


In [16]:
vals = model.predict(val_x)
mae(vals, val_y)

15.339275512524157

In [17]:
df_test = pd.read_csv("train/test.tsv", delimiter='\t', index_col=0)
predict = model.predict(df_test)

df_test["Cover_Type"] = predict
#df_test[["Vertical_Distance_To_Hydrology"]].to_csv("submit_test.csv")

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb

X = df_train.drop('Cover_Type', axis=1)
y = df_train['Cover_Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBMのモデルを設定
model = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100)

# モデルをトレーニング
model.fit(X_train, y_train)

# テストデータに対する予測
y_pred = model.predict(X_test)

# 予測結果の評価
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001088 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2261
[LightGBM] [Info] Number of data points in the train set: 185923, number of used features: 53
[LightGBM] [Info] Start training from score -1.008859
[LightGBM] [Info] Start training from score -0.717103
[LightGBM] [Info] Start training from score -2.801238
[LightGBM] [Info] Start training from score -5.348631
[LightGBM] [Info] Start training from score -4.096838
[LightGBM] [Info] Start training from score -3.511535
[LightGBM] [Info] Start training from score -3.345715
Accuracy: 82.40%


In [15]:
import pandas as pd

# テストデータの予測
y_pred = model.predict(X_test)

# 予測結果をDataFrameに保存
results_df = pd.DataFrame({
    'Index': X_test.index, 
    'Predicted_Cover_Type': y_pred
})
# インデックスを1から開始するように調整
results_df.index = range(1, len(results_df) + 1)

# データフレームの内容を確認
print(results_df.head())
# CSVファイルに書き出し（ヘッダーとインデックスは含めない）
results_df.to_csv('sample_submit.csv', index=False, header=False)

    Index  Predicted_Cover_Type
1   97293                     2
2  515976                     2
3  560531                     1
4     107                     1
5   34849                     2
